In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch import tensor

import warnings
from numpy import VisibleDeprecationWarning
warnings.filterwarnings(action='ignore', category=VisibleDeprecationWarning)


from generate_data import generate_data, data_loader

# onehot = pd.Series(data = [tensor([1,0,0]),tensor([0,1,0]),tensor([0,0,1])], index = ['a','b','c'])


In [2]:
batch_size = 1

lr = 0.0000001

loss_fn = nn.CrossEntropyLoss()


In [3]:
class LSTM_predictor(nn.Module):
    def __init__(self, SIZE):
        super(LSTM_predictor, self).__init__()

        self.SIZE = SIZE

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(3, SIZE)

        self.output = nn.Linear(SIZE, 2)

        self.activation = nn.Softmax()

        # self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, sentence):

        _, (lstm_out, _) = self.lstm(sentence)
        
        # print(lstm_out.shape)
        out = self.output(lstm_out[-1])
        # print(out.shape)
        # return self.softmax(out)


        # out = torch.sum(out,axis = 0)

        out = self.activation(out)

        return out
model = LSTM_predictor(16)

In [4]:
print(model)

LSTM_predictor(
  (lstm): LSTM(3, 16)
  (output): Linear(in_features=16, out_features=2, bias=True)
  (activation): Softmax(dim=None)
)


In [5]:
def test(model, X_test, y_test, min_length):
    model.eval()
    test_batches = data_loader(X_test,y_test,batch_size=batch_size, min_length = min_length)
    acc = 0.0
    i = 0
    for data, labels in test_batches:
        pred = model(data).view((-1,2))
        corr = (torch.argmax(pred,1) == torch.argmax(labels,1)).float().mean().item()
        acc += corr
        i += 1
    return acc/i

def train(model,n_epochs,lr, min_length):
    parameters = model.parameters()

    optimizer = torch.optim.Adam(parameters, lr=lr)

    model.train()

    losss, accs = [], []
    for i in range(n_epochs):
        X_train, y_train = generate_data("train", type = 2)
        X_validation, y_validation = generate_data("validation", type = 2)
        model.train()
        train_batches = data_loader(X_train,y_train,batch_size=batch_size, min_length = min_length)

        sum_loss = 0.0
        i_loss = 0
        for data, labels in train_batches:
            pred = model(data)
            loss_fn = F.binary_cross_entropy
            loss = loss_fn(pred,labels)
            loss.backward()
            optimizer.step()

            sum_loss += loss.item()
            i_loss += 1

        losss.append(sum_loss/i_loss)
        t = test(model,X_validation, y_validation, 70)
        accs.append(t)

        print(f"loss {sum_loss/i_loss:.3} - acc {t:.3}")

    return np.array(losss), np.array(accs)

# X_encoded[0]

In [6]:
N = 4*int(1e4)

min_length = 50

# X_train, y_train = generate_data(0.8, train_test_validation="train", type = 2)

# X_test, y_test = generate_data(0, train_test_validation="test", type = 2)
X_validation, y_validation = generate_data(train_test_validation="validation", type = 2)

In [7]:
# train(model, n_epochs = 10)
next(data_loader(X_validation,y_validation,batch_size=batch_size, min_length = min_length))

(tensor([[[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
         [[1., 0., 0.]],
 
 

In [8]:
def train_and_test(model, lr):
    N = 4*int(1e4)

    min_length = 20


    loss, acc = train(model, 200, lr, min_length = 20)
    
    min_length = 50

    X_test, y_test = generate_data("test", type = 2)

    t = test(model, X_test, y_test, min_length = 50)
    final_acc = test
    print(f"Final acc {t:.3}")

    return loss, acc, final_acc


lrs = [0.00001, 0.0005, 0.001]
sizes = [16, 32]


save_i = 0
for lr in lrs:
    for size in sizes:
        losss, accs, faccs = [], [], []
        for i in range(3):
            model = LSTM_predictor(size)
            print("Training with lr", lr,"and size", size)
            loss, acc, facc = train_and_test(model, lr)

            data = {"lr":lr,'size':size,'run':i,'loss':loss,'acc':acc,'facc':facc}

            # np.save(f"data{save_i}.npy",data, allow_pickle=True)
            save_i += 1



Training with lr 1e-05 and size 16


c:\Users\jakob\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


loss 0.694 - acc 0.5
loss 0.695 - acc 0.5
loss 0.693 - acc 0.5
loss 0.694 - acc 0.5
loss 0.695 - acc 0.5
loss 0.695 - acc 0.5
loss 0.694 - acc 0.5
loss 0.694 - acc 0.5
loss 0.694 - acc 0.5
loss 0.692 - acc 0.5
loss 0.692 - acc 0.5
loss 0.693 - acc 0.5
loss 0.691 - acc 0.5
loss 0.694 - acc 0.5
loss 0.693 - acc 0.5
loss 0.693 - acc 0.5
loss 0.694 - acc 0.5
loss 0.693 - acc 0.5
loss 0.693 - acc 0.5
loss 0.695 - acc 0.5
loss 0.694 - acc 0.55
loss 0.692 - acc 0.55
loss 0.693 - acc 0.55
loss 0.694 - acc 0.5
loss 0.693 - acc 0.55
loss 0.694 - acc 0.55
loss 0.693 - acc 0.5
loss 0.692 - acc 0.55
loss 0.693 - acc 0.55
loss 0.693 - acc 0.55
loss 0.693 - acc 0.5
loss 0.693 - acc 0.5
loss 0.692 - acc 0.5
loss 0.693 - acc 0.5
loss 0.692 - acc 0.5
loss 0.693 - acc 0.5
loss 0.692 - acc 0.5
loss 0.691 - acc 0.5
loss 0.691 - acc 0.5
loss 0.692 - acc 0.5


KeyboardInterrupt: 

In [ ]:
sum(range(21))

210